## Branch and Bound Algorithm for the Traveling Salesman Problem (based on matrix reduction method)
Implementation inspired by the following tutorial by Abdul Bari: https://www.youtube.com/watch?v=1FEP_sNb62k

To Dos:
- Change the starting node for a given instance and check how does the tree size varies according to the starting node (done)
- Visualize tree with networkx (would be too long for instances greater than some 5 nodes)

In [3]:
import numpy as np
import pandas as pd
import time
import random

In [4]:
# Random node creation
def create_nodes(num_of_nodes):
    #global node_loc
    node_loc = []
    for i in range(0,num_of_nodes):
        x = np.random.randint(0,100)
        y = np.random.randint(0,100)
        node_loc.append(list([x,y]))
    return node_loc
        

# Function for distance calculation between nodes
def func_dist_calc(node_loc):
    dist_list = []
    for i in range(0,len(node_loc)):
        for j in range(0,len(node_loc)):
            dist = ((node_loc[i][0]-node_loc[j][0])**2 + (node_loc[i][1]-node_loc[j][1])**2)**(1/2)
            dist = np.nan if dist == 0 else dist
            dist_list.append(dist)
    return dist_list

In [5]:
def func_matrix_reduced(matrix):
    min_val_rows = []
    for e,i in enumerate(matrix):
        min_val = np.nanmin(i)
        min_val_rows.append(0 if np.isnan(min_val)==True else min_val)
        matrix[e] = matrix[e]-min_val

    matrix = matrix.transpose()
    min_val_cols = []
    for e,i in enumerate(matrix):
        min_val = np.nanmin(i)
        min_val_cols.append(0 if np.isnan(min_val)==True else min_val)
        matrix[e] = matrix[e]-min_val

    matrix = matrix.transpose()
    reduced_cost = (np.sum(min_val_rows) + np.sum(min_val_cols))
    return matrix, reduced_cost

In [6]:
def func_matrix_preprocessing(matrix, from_node, to_node):
    matrix[from_node] = np.nan
    matrix.transpose()[to_node] = np.nan
    # Blocking the path for returning to the origin
    matrix[to_node][0] = np.nan
    return matrix

In [7]:
# Binary search: find the correct position to insert the node with the least cost in the queue to be next explored
def binary_search(num_to_search, queue):
    if len(queue)==0:
        insert_pos=0
    else:
        queue_temp = queue.copy()
        index = [i for i in range(0,len(queue_temp))]
        while len(queue_temp)>1:
            pos = int(np.floor(len(queue_temp)/2))
            if num_to_search < queue_temp[pos]:
                queue_temp = queue_temp[0:pos]
                index = index[0:pos]
            else:
                queue_temp = queue_temp[pos:]
                index = index[pos:]
        insert_pos = 0 if num_to_search<queue[0] else int(index[0]+1)
    return insert_pos

In [8]:
def func_calculate_tour_distance(df_instance, tour_path):
    distance = 0
    for e,i in enumerate(tour_path):
        if e+1<len(tour_path):
            distance += df_instance.loc[i,tour_path[e+1]]
        else:
            distance += df_instance.loc[i,tour_path[0]]
    return distance

In [9]:
def LeastCost_BranchnBound(instance):
    global_upper = np.inf
    tree, queue_full, queue_cost = [], [], [0]
    start_process, node_number, level = 1, 0, 0
    instance_size = len(instance)
    instance_nodes = [i for i in range(0,len(instance))]

    # Iterates until the queue containing the nodes to explore is empty
    while len(queue_cost)>0:
    #for i in range(0,9):
        if start_process==1:
            # First reduced matrix without matrix preprocessing
            queue_cost.clear()
            parent_matrix, parent_matrix_cost = func_matrix_reduced(instance.copy())
            tree.append([node_number, level, 0, 0, parent_matrix, parent_matrix_cost, [0]])
            queue_full.append([node_number, level, 0, 0, parent_matrix, parent_matrix_cost, [0]])
            queue_cost.append(parent_matrix_cost)
            start_process-=1
            node_number+=1
            #print(global_upper)

        else:
            if queue_full[0][5] <= global_upper:
                # Retrieving the best node found on the queue, best objective values will be first and worst last
                curr_node = queue_full[0]
                queue_full.pop(0)
                queue_cost.pop(0)      

            if len(curr_node[6])<len(instance):
                # Defining current variables, same which are taken from most recent value on the queue, namely the parent node 
                matrix_nodes_to_branch = [i for i in instance_nodes if i not in curr_node[6]]
                level = curr_node[1]+1
                from_node = curr_node[3]
                parent_matrix = curr_node[4]
                parent_matrix_cost = curr_node[5]
                parent_solution = curr_node[6]
                for to_node in matrix_nodes_to_branch:
                    # Creating the new reduced matrix, one for each new child
                    child_matrix = func_matrix_preprocessing(parent_matrix.copy(), from_node, to_node)
                    child_matrix, child_matrix_cost = func_matrix_reduced(child_matrix)
                    child_node_cost = parent_matrix_cost + parent_matrix[from_node][to_node] + (0 if np.isnan(child_matrix_cost)==True else child_matrix_cost)

                    full_child_node = [node_number, level, from_node, to_node, child_matrix, child_node_cost, (parent_solution+[to_node])]
                    tree.append(full_child_node)
                    # Obtaining the position in the queue 
                    queue_insert_pos = binary_search(child_node_cost, queue_cost)
                    # Inserting on the given position on both queues, one with the whole history and second the one used to explore
                    queue_full.insert(queue_insert_pos, full_child_node)
                    queue_cost.insert(queue_insert_pos, child_node_cost)
                    # Updating the best known result (incumbent)
                    global_upper = child_node_cost if (child_node_cost < global_upper and len((parent_solution+[to_node]))==len(instance)) else global_upper
                    node_number+=1
                    #print(global_upper)
            else:
                while len(queue_cost)>0:
                    queue_full.pop(0)
                    queue_cost.pop(0)
                    
    return tree

In [10]:
# Instance depicted in Abdul Bari´s tutorial
instance = np.array([[np.nan, 20, 30, 10, 11],
                     [15, np.nan, 16, 4, 2],
                     [3, 5, np.nan, 2, 4],
                     [19, 6, 18, np.nan, 3],
                     [16, 4, 7, 16, np.nan],
                    ])

In [12]:
node_loc = create_nodes(12)
instance = func_dist_calc(node_loc)
instance = np.reshape(instance, (len(node_loc),len(node_loc)))

By creating a for loop and randomly shuffling the node, we would like to ensure that we obtain the same solution regardless the starting point and the order of the nodes on the tree. Of course size of the tree and solve time could vary in the for loop.

In [16]:
node_loc = create_nodes(12)

# Instance used for the below example and explanation, alternatively for a new instance use the generator above "node_loc = create_nodes(n_nodes)"
node_loc  = [[74, 72],[15, 76],[94, 98],[60, 71],[46, 3],[22, 62],[16, 33],[95, 83],[61, 98],[22, 30],[19, 19],[93, 39]]

for i in range(0,len(node_loc)):
    random.shuffle(node_loc)
    instance = func_dist_calc(node_loc)
    instance = np.reshape(instance, (len(node_loc),len(node_loc)))
    df_instance = pd.DataFrame(instance)
    
    tree = LeastCost_BranchnBound(instance)
    solution = tree[-1]
    solution_cost = solution[5]
    tour_path = solution[-1]
    distance_calc = func_calculate_tour_distance(df_instance, tour_path)
    
    print("Iteration no: ", i, " | Length of B&B tree: ", len(tree),
         "\n Solution cost: ", "{:.2f}".format(solution_cost), "Distance calculated: ", "{:.2f}".format(distance_calc)
         )

C:\Users\JonathanA\anaconda3\envs\scip\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\Users\JonathanA\anaconda3\envs\scip\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: All-NaN slice encountered
  # This is added back by InteractiveShellApp.init_path()


Iteration no:  0  | Length of B&B tree:  3354 
 Solution cost:  334.42 Distance calculated:  334.42
Iteration no:  1  | Length of B&B tree:  3360 
 Solution cost:  334.42 Distance calculated:  334.42
Iteration no:  2  | Length of B&B tree:  3763 
 Solution cost:  334.42 Distance calculated:  334.42
Iteration no:  3  | Length of B&B tree:  1377 
 Solution cost:  334.42 Distance calculated:  334.42
Iteration no:  4  | Length of B&B tree:  3860 
 Solution cost:  353.55 Distance calculated:  334.35
Iteration no:  5  | Length of B&B tree:  2768 
 Solution cost:  334.42 Distance calculated:  334.42
Iteration no:  6  | Length of B&B tree:  2716 
 Solution cost:  429.39 Distance calculated:  278.60
Iteration no:  7  | Length of B&B tree:  3589 
 Solution cost:  334.42 Distance calculated:  334.42
Iteration no:  8  | Length of B&B tree:  1297 
 Solution cost:  334.42 Distance calculated:  266.79
Iteration no:  9  | Length of B&B tree:  2773 
 Solution cost:  334.42 Distance calculated:  324.99


As seen on the above log, on the `"solution cost"` column, we can see that unfortunately the solution cost (objective value) is not the same for all the instances variations.  Specifically by taking a look at `Iterations no. 4, 6 and 10`, we can see that their cost is greater than the rest of the iterations, which yield the optimal solution. 

Unfortunately my approach does not scale well, and the algorithm starts to struggle with problem sizes as small as 15 nodes.

Due to time constraints I could not further dig into both problems stated above.

## TSP SCIP
based on the implementation at: https://github.com/scipopt/PySCIPOpt/blob/master/tests/test_tsp.py

Here we would like to compare our solution with that of the non-commercial solver SCIP.

In [17]:
from pyscipopt import Model, Conshdlr, quicksum, SCIP_RESULT

import pytest

itertools = pytest.importorskip("itertools")
networkx = pytest.importorskip("networkx")

EPS = 1.e-6


# subtour elimination constraint handler
class TSPconshdlr(Conshdlr):

    def __init__(self, variables):
        self.variables = variables

    # find subtours in the graph induced by the edges {i,j} for which x[i,j] is positive
    # at the given solution; when solution is None, then the LP solution is used
    def find_subtours(self, solution = None):
        edges = []
        x = self.variables
        for (i, j) in x:
            if self.model.getSolVal(solution, x[i, j]) > EPS:
                edges.append((i, j))

        G = networkx.Graph()
        G.add_edges_from(edges)
        components = list(networkx.connected_components(G))

        if len(components) == 1:
            return []
        else:
            return components

    # checks whether solution is feasible, ie, if there are no subtours;
    # since the checkpriority is < 0, we are only called if the integrality
    # constraint handler didn't find infeasibility, so solution is integral
    def conscheck(self, constraints, solution, check_integrality,
                  check_lp_rows, print_reason, completely, **results):
        if self.find_subtours(solution):
            return {"result": SCIP_RESULT.INFEASIBLE}
        else:
            return {"result": SCIP_RESULT.FEASIBLE}

    # enforces LP solution
    def consenfolp(self, constraints, n_useful_conss, sol_infeasible):
        subtours = self.find_subtours()
        if subtours:
            x = self.variables
            for subset in subtours:
                self.model.addCons(quicksum(x[i, j] for(i, j) in pairs(subset))
                                   <= len(subset) - 1)
                print("cut: len(%s) <= %s" % (subset, len(subset) - 1))
            return {"result": SCIP_RESULT.CONSADDED}
        else:
            return {"result": SCIP_RESULT.FEASIBLE}

    def conslock(self, constraint, locktype, nlockspos, nlocksneg):
        pass


# builds tsp model; adds variables, degree constraint and the subtour elimination constaint handler
def create_tsp(vertices, distance):
    model = Model("TSP")

    x = {}  # binary variable to select edges
    for (i, j) in pairs(vertices):
        x[i, j] = model.addVar(vtype = "B", name = "x(%s,%s)" % (i, j))

    for i in vertices:
        model.addCons(quicksum(x[j, i] for j in vertices if j < i) +
                      quicksum(x[i, j] for j in vertices if j > i) == 2, "Degree(%s)" % i)

    conshdlr = TSPconshdlr(x)

    model.includeConshdlr(conshdlr, "TSP", "TSP subtour eliminator",
                          chckpriority = -10, needscons = False)
    model.setBoolParam("misc/allowstrongdualreds", False)

    model.setObjective(quicksum(distance[i, j] * x[i, j] for (i, j) in pairs(vertices)),
                       "minimize")

    return model, x


def solve_tsp(vertices, distance):
    model, x = create_tsp(vertices, distance)
    model.optimize()

    edges = []
    for (i, j) in x:
        if model.getVal(x[i, j]) > EPS:
            edges.append((i, j))

    return model.getObjVal(), edges


# returns all undirected edges between vertices
def pairs(vertices):
    return itertools.combinations(vertices, 2)


def test_main(instance_scip):
    vertices = [i+1 for i in range(0,len(node_loc))]
    
    instance_scip = np.reshape(instance_scip, (len(node_loc)*len(node_loc),))
    distance = {}
    for r in range(0,len(node_loc)):
        for c in range(0,len(node_loc)):
            if r!=c:
                distance[(r+1,c+1)] = instance_scip[(c*len(node_loc) + r)]

    objective_value, edges = solve_tsp(vertices, distance)

    print("Optimal tour:", edges)
    print("Optimal cost:", objective_value)


if __name__ == "__main__":
    test_main(instance)

Optimal tour: [(1, 4), (1, 9), (2, 4), (2, 6), (3, 8), (3, 9), (5, 11), (5, 12), (6, 7), (7, 10), (8, 12), (10, 11)]
Optimal cost: 334.4245756465901


In [18]:
instance_scip = instance.copy()

vertices = [i+1 for i in range(0,len(node_loc))]
    
instance_scip = np.reshape(instance_scip, (len(node_loc)*len(node_loc),))
distance = {}
for r in range(0,len(node_loc)):
    for c in range(0,len(node_loc)):
        if r!=c:
            distance[(r+1,c+1)] = instance_scip[(c*len(node_loc) + r)]

## B&B for the TSP Geeks for Geeks
A further benchmark taken from: https://www.geeksforgeeks.org/traveling-salesman-problem-using-branch-and-bound-2/

In [19]:
# Python3 program to solve Traveling Salesman Problem using Branch and Bound.
import math
maxsize = float('inf')
  
# Function to copy temporary solution
# to the final solution
def copyToFinal(curr_path):
    final_path[:N + 1] = curr_path[:]
    final_path[N] = curr_path[0]

# Function to find the minimum edge cost 
# having an end at the vertex i
def firstMin(adj, i):
    min = maxsize
    for k in range(N):
        if adj[i][k] < min and i != k:
            min = adj[i][k]
  
    return min
  
# function to find the second minimum edge 
# cost having an end at the vertex i
def secondMin(adj, i):
    first, second = maxsize, maxsize
    for j in range(N):
        if i == j:
            continue
        if adj[i][j] <= first:
            second = first
            first = adj[i][j]
  
        elif(adj[i][j] <= second and 
             adj[i][j] != first):
            second = adj[i][j]
  
    return second
  
# function that takes as arguments:
# curr_bound -> lower bound of the root node
# curr_weight-> stores the weight of the path so far
# level-> current level while moving
# in the search space tree
# curr_path[] -> where the solution is being stored
# which would later be copied to final_path[]
def TSPRec(adj, curr_bound, curr_weight, 
              level, curr_path, visited):
    global final_res
      
    # base case is when we have reached level N 
    # which means we have covered all the nodes once
    if level == N:
          
        # check if there is an edge from
        # last vertex in path back to the first vertex
        if adj[curr_path[level - 1]][curr_path[0]] != 0:
              
            # curr_res has the total weight
            # of the solution we got
            curr_res = curr_weight + adj[curr_path[level - 1]]\
                                        [curr_path[0]]
            if curr_res < final_res:
                copyToFinal(curr_path)
                final_res = curr_res
        return
  
    # for any other level iterate for all vertices
    # to build the search space tree recursively
    for i in range(N):
          
        # Consider next vertex if it is not same 
        # (diagonal entry in adjacency matrix and 
        #  not visited already)
        if (adj[curr_path[level-1]][i] != 0 and
                            visited[i] == False):
            temp = curr_bound
            curr_weight += adj[curr_path[level - 1]][i]
  
            # different computation of curr_bound 
            # for level 2 from the other levels
            if level == 1:
                curr_bound -= ((firstMin(adj, curr_path[level - 1]) + 
                                firstMin(adj, i)) / 2)
            else:
                curr_bound -= ((secondMin(adj, curr_path[level - 1]) +
                                 firstMin(adj, i)) / 2)
  
            # curr_bound + curr_weight is the actual lower bound 
            # for the node that we have arrived on.
            # If current lower bound < final_res, 
            # we need to explore the node further
            if curr_bound + curr_weight < final_res:
                curr_path[level] = i
                visited[i] = True
                  
                # call TSPRec for the next level
                TSPRec(adj, curr_bound, curr_weight, 
                       level + 1, curr_path, visited)
  
            # Else we have to prune the node by resetting 
            # all changes to curr_weight and curr_bound
            curr_weight -= adj[curr_path[level - 1]][i]
            curr_bound = temp
  
            # Also reset the visited array
            visited = [False] * len(visited)
            for j in range(level):
                if curr_path[j] != -1:
                    visited[curr_path[j]] = True

# This function sets up final_path
def TSP(adj):
      
    # Calculate initial lower bound for the root node 
    # using the formula 1/2 * (sum of first min + 
    # second min) for all edges. Also initialize the 
    # curr_path and visited array
    curr_bound = 0
    curr_path = [-1] * (N + 1)
    visited = [False] * N
  
    # Compute initial bound
    for i in range(N):
        curr_bound += (firstMin(adj, i) + 
                       secondMin(adj, i))
  
    # Rounding off the lower bound to an integer
    curr_bound = math.ceil(curr_bound / 2)
  
    # We start at vertex 1 so the first vertex 
    # in curr_path[] is 0
    visited[0] = True
    curr_path[0] = 0
  
    # Call to TSPRec for curr_weight 
    # equal to 0 and level 1
    TSPRec(adj, curr_bound, 0, 1, curr_path, visited)

    
# Driver code  

# Adjacency matrix for the given graph
adj = [[0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]]

adj = np.array([[0, 10, 15, 20],
               [10, 0, 35, 25],
               [15, 35, 0, 30],
               [20, 25, 30, 0]])

N = len(adj)

instance_GfG = instance_scip.copy()
instance_GfG = np.reshape(np.array([i if np.isnan(i)==False else 0 for i in instance_GfG]),(len(instance),len(instance)))
N = len(instance_GfG)
  
# final_path[] stores the final solution 
# i.e. the // path of the salesman.
final_path = [None] * (N + 1)
  
# visited[] keeps track of the already
# visited nodes in a particular path
visited = [False] * N
  
# Stores the final minimum weight
# of shortest tour.
final_res = maxsize
  
TSP(instance_GfG)
  
print("Minimum cost :", final_res)
print("Path Taken : ", end = ' ')
for i in range(N + 1):
    print(final_path[i], end = ' ')

# This code is contributed by ng24_7

Minimum cost : 334.4245756465901
Path Taken :  0 3 1 5 6 9 10 4 11 7 2 8 0 